# Fiddler examples have moved! [Deprecation Notice]

Dear user thank you for using fiddler product, we appreciate your time! We have moved the examples to a new github repo located at the following link


***
# [New fiddler-examples repo](https://github.com/fiddler-labs/fiddler-examples)
***

# Fiddler-Snowflake Integration 2 - Event Publishing

This notebook is used in conjuction with the guide on how to use Fiddler for you machine learning model data stored in Snowflake. Link

In this notebook we will look at how we can use data stored in Snowflake to upload to fiddler as baseline data for your ML model. We will also register your model on the Fiddler platform.

[Link](https://colab.research.google.com/github/fiddler-labs/fiddler-samples/blob/master/content_root/tutorial/integration-examples/snowflake/Fiddler-Snowflake%20Integration%20-%20Model%20Registration.ipynb) to part 1 of the guide on how to upload baseline data and register model on Fiddler.

## Establish connection to Snowflake

Install required libraries to loading data from Snowflake

In [ ]:
!pip install snowflake

In [ ]:
# Imports
import pandas as pd
from snowflake import connector
import getpass

In [ ]:
# Enter your Snowflake credentials
snowflake_username = ''
snowflake_account = ''
snowflake_role = ''
snowflake_warehouse = ''

In [ ]:
# Enter your Snowflake password
snowflake_password = getpass.getpass("Enter password:")

In [ ]:
# establish Snowflake connection
connection = connector.connect(user=snowflake_username, 
                               password=snowflake_password, 
                               account=snowflake_account, 
                               role=snowflake_role, 
                               warehouse=snowflake_warehouse
                              )

Below we use the query to load the required data from Snowflake table

In [ ]:
# sample SQL query
sql_query = 'select * from FIDDLER.FIDDLER_SCHEMA.CHURN_PRODUCTION LIMIT 100'

In [ ]:
# create cursor object
cursor = connection.cursor()

# execute SQL query inside Snowflake
cursor.execute(sql_query)

store the query results as a pandas DataFrame

In [ ]:
production_df = cursor.fetch_pandas_all()

In [ ]:
# print dataset details
print('Dataset Dimensions:', production_df.shape)
print('Columns:', production_df.columns)
production_df.head()

## Publish data to Fiddler

### Imports

We will publish the Snowflake data that we have just imported, on the Fiddler platform.

If you are new to Fiddler and want a detailed explanation on how to setup the fiddler environment please refer to our Getting Started Guide.

In [ ]:
!pip install -q fiddler-client;

In [ ]:
import numpy as np
import pandas as pd
import fiddler as fdl

print(f"Running client version {fdl.__version__}")

### Connect to Fiddler

In [ ]:
URL = #
ORG_ID = #
AUTH_TOKEN = #

In [ ]:
# Initiate Fiddler client
client = fdl.FiddlerApi(
    url=URL,
    org_id=ORG_ID,
    auth_token=AUTH_TOKEN
)

Convert the pandas DataFrame into a dictionary.

In [ ]:
production_events = production_df.to_dict(orient='records')
production_events[0]

You can use the client's `publish_event` function to start pumping data into Fiddler!
  
*Just include:*
1. The event dictionary
2. A timestamp for when the event occurred

In [ ]:
from tqdm import tqdm

for event in tqdm(production_events):

    client.publish_event(
        project_id=PROJECT_ID,
        model_id=MODEL_ID,
        event=event,
        event_timestamp=event['timestamp']
    )

Now that we have events published to Fiddler, we can access the URL below to monitor the publish events

In [ ]:
print('/'.join([URL, 'projects', PROJECT_ID, 'models', MODEL_ID, 'monitor']))